In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from scipy import ndimage
import xml
import xml.etree.ElementTree as ET
import cv2
import warnings
from skimage.transform import resize

In [2]:
# Okay, what needs to happen here?

# Let's consider a single block
    # - We have 20 time points (therefore images - just the horizontal ones)
    # - Then we need to get the convolution maps 
    # - Next, we create the subwindows - 910 per image, and sum up the convolution to get the density
    # - Each image will output subwindowed densities of shape (910, ) or (910, 1) - we might prefer the latter
    # - Now since we have 20 images/convolved maps in the sequence, we will stack the final result to be of shape (910, 20, 1)
    # - We can then store these np arrays and later stack these along with the feature maps to create the 910 csv files that will then be used for the BLAR

In [3]:
# Okay, let's start with the creation of density maps

In [4]:
# Also do sanity checks to verify the counts

In [5]:
# We have previously written functions, let's use them here?

In [6]:
# Choose the jpeg and xml files for which we need the density maps - basically all the horizontal images (20 of them) in the test blocks

In [7]:
def chose_xml_and_jpeg(file_location):
    # list all files in location
    list_of_all_files = os.listdir(file_location)
    # sort files
    list_of_all_files.sort()
    # separate xml and jpeg files
    all_xml_files = [file for file in list_of_all_files if file.split('.')[-1] == 'xml']
    all_xml_files.sort()
    all_jpeg_files = [file for file in list_of_all_files if file not in all_xml_files]
    all_jpeg_files.sort()
    # get the final 20 files
    chosen_xml_files = all_xml_files[-20:]
    chosen_jpeg_files = all_jpeg_files[-20:]
    # make sure the xml and jpeg files correspond to each other?
    mean = np.mean([file.split('.')[0] for file in chosen_xml_files] == [file.split('.')[0] for file in chosen_jpeg_files])

    # chose the required files only - notice that for the inputs we do not need to create their density maps - therefore we do not need the xml files
    task_specific_image_files = chosen_jpeg_files[-7:]
    task_specific_xml_files =chosen_xml_files[-7:]
    return(chosen_jpeg_files, chosen_xml_files, mean)

In [8]:
# Get the convolved density maps

In [9]:
# for horizontally annotated images
def get_density_maps_horizontal(file_name, image_path, xml_path):
    xml_file = file_name + '.xml'
    xml_file_path = os.path.join(xml_path, xml_file)

    # Get coords from the xml file
    # parse the xml file
    parsed_file = ET.parse(xml_file_path)
    # get the roots
    root = parsed_file.getroot()
    # get the roots here
    coords = []
    for child in root:
        for i in child:
            for j in i:
                coords.append(int(j.text))
    
    # chunk the points into sets of 4 - these are the coordinates of the bounding boxes
    points_tupples = []
    for i in range(0, len(coords), 4):
        points_tupples.append(coords[i:i + 4])

    # make a dataframe with these points
    coords_df = pd.DataFrame(points_tupples, columns = ["bleft_x", "bleft_y", "tright_x", "tright_y"])

    # compute the number of tassels in each image
    no_of_tassels = len(points_tupples)

    # compute the mid coordinates
    coords_df["mid_x"] = (round(0.5*(coords_df["bleft_x"] + coords_df["tright_x"]))).astype(int)
    coords_df["mid_y"] = (round(0.5*(coords_df["bleft_y"] + coords_df["tright_y"]))).astype(int)

    # extract the mid cordinates
    mid_coords = coords_df[["mid_x", "mid_y"]]
    # cap the coords at the max height and width values
    mid_coords.loc[mid_coords['mid_x'] > 1024, 'mid_x'] = 1023
    mid_coords.loc[mid_coords['mid_y'] > 768, 'mid_y'] = 767
    warnings.filterwarnings("ignore")

    # plot the bounding boxes on images
    # get image name and path
    image_name = file_name + '.jpeg'
    imge_file_path = os.path.join(image_path, image_name)
    # read the image
    read_image = plt.imread(imge_file_path)
    read_image = ndimage.rotate(read_image, 0)

    # check the shape of the read image
    read_image_shape = read_image.shape
    #  plot the bounding boxes on the image
    for points in points_tupples:
        annotated_image = cv2.rectangle(read_image, (points[0],points[1]), (points[2],points[3]), color = (255,0,0), thickness = 2)
    # plt.figure(figsize = (12,18))
    plt.imshow(annotated_image)
    plt.show()

    # plot the mid points on the image
    coords_list = mid_coords.values.tolist()
    # read the image again
    read_image_again = plt.imread(imge_file_path)
    read_image_again = ndimage.rotate(read_image_again, 0)
    # draw the circles on image
    for i in coords_list:
        image_with_mids = cv2.circle(read_image_again, i, radius=5, color=(255, 0, 0), thickness=-1)
    # look at the annotated image
    # plt.figure(figsize = (12,18))
    plt.imshow(image_with_mids)
    plt.show()

    # also try creating the density map here
    # first create the empty maps
    np_image = np.zeros((read_image_shape[0], read_image_shape[1]))
    # get the dot maps
    for point in coords_list:
        np_image[point[1], point[0]] = 1
    # plot the image
    # plt.figure(figsize = (12,18))
    plt.imshow(np_image, cmap = "Greys")
    plt.show()

    # now define the kernel and run the convolution
    one_d_kerenel = cv2.getGaussianKernel(50,5)
    two_d_kernel = np.multiply(one_d_kerenel.T, one_d_kerenel)

    # Shape of the 2D kernel
    twoD_shape = two_d_kernel.shape
        
    # do the convolution
    convolution = ndimage.convolve(np_image, two_d_kernel)
        
    # plot the density map
    # plt.figure(figsize = (12,18))
    plt.imshow(convolution, cmap = "Greys")
    plt.show()
        
    # get the sums of the images
    img_sum = np.sum(convolution)

    return(file_name, read_image_shape, no_of_tassels, img_sum, convolution)

In [10]:
# I think the no_of_tassels and img_sum will give us the same numbers - and we may need to store these for future use as the true densities - maybe these will be needed for the inference in the BLAR model

In [11]:
# Get the sub-window-wise densities

In [12]:
def create_densities_for_subwindows(density_map, stride = 30, kernel_size = 30):
        
    # create the counts for the subwindows as follows
    img_height = density_map.shape[0]
    img_width = density_map.shape[1]
    
    density_sums = []
    for i in  range(0, img_height, stride):
        for j in range(0, img_width, stride):
            # sub_window = loaded_im_file[i: i + kernel_size, j : j + kernel_size,:]
            density = density_map[i: i + kernel_size, j : j + kernel_size]
            dense_sum = np.sum(density)
            density_sums.append(dense_sum)

    print("sum of the convolved map: ", np.sum(density_sums))
    return density_sums